<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Capstone Project is part of the 5-Day AI Agents Intensive Course with Google (Nov 10 - 14, 2025)</p>
</div>


<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Install Agent Development Kit (ADK)</p>
</div>


In [32]:
!pip install -q google-adk google-genai

import os
import json
from datetime import datetime, timedelta
from typing import Any, Dict, List, Tuple

from google.adk.agents import LlmAgent  # core LLM agent
from google.adk.models.google_llm import Gemini
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.runners import InMemoryRunner
from google.genai import types

from kaggle_secrets import UserSecretsClient
import asyncio

<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Configure your Gemini API Key</p>
</div>

In [33]:
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_Key")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [34]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

DEFAULT_MODEL = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

APP_NAME_LOGGING = "habit_logging_app"
APP_NAME_INSIGHT = "habit_insight_app"
APP_NAME_PLANNER = "habit_planner_app"

USER_ID = "user123"

<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Data Loading, Preprocessing and helper functions</p>
</div>


In [35]:
HABIT_DB_FILE = "habit_database.json"
GOALS_DB_FILE = "habit_goals.json"


def _load_json(path: str) -> Dict[str, Any]:
    try:
        with open(path, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}
    except json.JSONDecodeError:
        print(f"Warning: invalid JSON in {path}, resetting.")
        return {}


def _save_json(path: str, data: Dict[str, Any]) -> None:
    with open(path, "w") as f:
        json.dump(data, f, indent=2)


def _load_habit_db() -> Dict[str, Any]:
    return _load_json(HABIT_DB_FILE)


def _save_habit_db(db: Dict[str, Any]) -> None:
    _save_json(HABIT_DB_FILE, db)


def _load_goals_db() -> Dict[str, Any]:
    return _load_json(GOALS_DB_FILE)


def _save_goals_db(db: Dict[str, Any]) -> None:
    _save_json(GOALS_DB_FILE, db)


<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Custom functions and tools</p>
</div>


In [36]:

# ======================
# TOOLS – shared actions for all agents
# ======================


def add_habit_entry(
    tool_context: ToolContext,
    date: str,
    habit: str,
    category: str,
    duration_minutes: int,
    rating: int,
    energy_level: int,
    notes: str,
) -> str:
    """
    Store a structured habit entry for a given date.

    Example entry shape:
    {
      "habit": "workout",
      "category": "health",
      "duration_minutes": 30,
      "rating": 4,
      "energy_level": 3,
      "notes": "cardio at home"
    }
    """
    db = _load_habit_db()
    user_db = db.get(USER_ID, {})

    day_entries = user_db.get(date, [])
    day_entries.append(
        {
            "habit": habit,
            "category": category,
            "duration_minutes": duration_minutes,
            "rating": rating,
            "energy_level": energy_level,
            "notes": notes,
        }
    )
    user_db[date] = day_entries
    db[USER_ID] = user_db
    _save_habit_db(db)

    return (
        f"Logged habit '{habit}' for {date}: "
        f"category={category}, duration={duration_minutes} min, "
        f"rating={rating}, energy={energy_level}, notes='{notes}'"
    )


# Habit retrieval tools

def get_entries_by_date(tool_context: ToolContext, date: str) -> str:
    """
    Return a readable summary of all entries for a specific date.
    """
    db = _load_habit_db()
    user_db = db.get(USER_ID, {})
    day_entries = user_db.get(date, [])

    if not day_entries:
        return f"No habit entries found for {date}."

    lines = [f"Habit entries for {date}:"]
    for e in day_entries:
        lines.append(
            f"- {e['habit']} ({e['category']}): "
            f"{e.get('duration_minutes')} min, "
            f"rating={e.get('rating')}, energy={e.get('energy_level')}, "
            f"notes={e.get('notes')}"
        )
    return "\n".join(lines)


def get_entries_last_n_days(tool_context: ToolContext, n_days: int) -> str:
    """
    Return entries for the last n_days (including today) in a readable format.
    """
    db = _load_habit_db()
    user_db = db.get(USER_ID, {})

    today = datetime.today().date()
    result_lines: List[str] = []

    for i in range(n_days):
        day = today - timedelta(days=i)
        key = day.isoformat()
        entries = user_db.get(key, [])
        result_lines.append(f"Date: {key}")
        if not entries:
            result_lines.append("  (no entries)")
        else:
            for e in entries:
                result_lines.append(
                    f"  - {e['habit']} ({e['category']}): "
                    f"{e.get('duration_minutes')} min, "
                    f"rating={e.get('rating')}, energy={e.get('energy_level')}, "
                    f"notes={e.get('notes')}"
                )
        result_lines.append("")

    return "\n".join(result_lines).strip()


def list_all_habits(tool_context: ToolContext) -> str:
    """
    List all unique habits the user has logged.
    """
    db = _load_habit_db()
    user_db = db.get(USER_ID, {})
    habits = set()

    for entries in user_db.values():
        for e in entries:
            habits.add(e["habit"])

    if not habits:
        return "No habits logged yet."

    return "Tracked habits:\n" + "\n".join(f"- {h}" for h in sorted(habits))


# Goal-setting tools

def set_goal(
    tool_context: ToolContext,
    habit: str,
    period: str,
    metric: str,
    target_value: float,
) -> str:
    """
    Set a goal for a habit.

    Examples:
    - habit="workout", period="week", metric="sessions", target_value=4
    - habit="study", period="week", metric="minutes", target_value=600
    """
    goals = _load_goals_db()
    user_goals = goals.get(USER_ID, {})

    user_goals[habit] = {
        "period": period,          # "day", "week", "month", "year"
        "metric": metric,          # "sessions" or "minutes"
        "target_value": target_value,
    }

    goals[USER_ID] = user_goals
    _save_goals_db(goals)

    return f"Set goal for '{habit}': {target_value} {metric} per {period}."


def get_goals(tool_context: ToolContext) -> str:
    """
    Return all goals for the current user as pretty-printed JSON string.
    """
    goals = _load_goals_db()
    user_goals = goals.get(USER_ID, {})
    if not user_goals:
        return "No goals set yet."

    return json.dumps(user_goals, indent=2)


# Analysis tools

def _aggregate_habits_over_range(
    user_db: Dict[str, Any],
    from_date: datetime.date,
    to_date: datetime.date,
) -> Dict[str, Dict[str, Any]]:
    """
    Aggregate habit stats over [from_date, to_date], inclusive.
    Returns a dict keyed by habit name with fields:
      sessions, total_minutes, ratings[], energies[]
    """
    stats: Dict[str, Dict[str, Any]] = {}
    d = from_date
    while d <= to_date:
        key = d.isoformat()
        entries = user_db.get(key, [])
        for e in entries:
            habit = e["habit"]
            s = stats.setdefault(
                habit,
                {
                    "sessions": 0,
                    "total_minutes": 0,
                    "ratings": [],
                    "energies": [],
                },
            )
            s["sessions"] += 1
            if isinstance(e.get("duration_minutes"), (int, float)):
                s["total_minutes"] += e["duration_minutes"]
            if isinstance(e.get("rating"), (int, float)):
                s["ratings"].append(e["rating"])
            if isinstance(e.get("energy_level"), (int, float)):
                s["energies"].append(e["energy_level"])
        d += timedelta(days=1)
    return stats


def analyze_progress_against_goals(
    tool_context: ToolContext,
    n_days: int,
) -> str:
    """
    Analyze user's progress vs goals for the last n_days.

    Returns JSON with structure:
    {
      "window_days": 7,
      "per_habit": {
        "workout": {
          "goal": "4 sessions/week (target 4.0)",
          "actual_sessions": 3,
          "actual_minutes": 90,
          "consistency_score": 0.75,
          "avg_rating": 4.3,
          "avg_energy": 3.5
        },
        ...
      }
    }
    """
    db = _load_habit_db()
    user_db = db.get(USER_ID, {})
    goals_db = _load_goals_db()
    user_goals = goals_db.get(USER_ID, {})

    today = datetime.today().date()
    from_date = today - timedelta(days=n_days - 1)
    stats = _aggregate_habits_over_range(user_db, from_date, today)

    result: Dict[str, Any] = {"window_days": n_days, "per_habit": {}}

    for habit, goal_info in user_goals.items():
        period = goal_info["period"]
        metric = goal_info["metric"]  # "sessions" or "minutes"
        target = float(goal_info["target_value"])
        habit_stats = stats.get(habit, None)

        actual_sessions = habit_stats["sessions"] if habit_stats else 0
        actual_minutes = habit_stats["total_minutes"] if habit_stats else 0

        if habit_stats and habit_stats["ratings"]:
            avg_rating = sum(habit_stats["ratings"]) / len(habit_stats["ratings"])
        else:
            avg_rating = None

        if habit_stats and habit_stats["energies"]:
            avg_energy = sum(habit_stats["energies"]) / len(habit_stats["energies"])
        else:
            avg_energy = None

        if metric == "sessions":
            actual_value = actual_sessions
        else:
            actual_value = actual_minutes

        # Rough consistency: actual_value / (target * weeks_in_window)
        # For simplicity assume window ~1 period (week/month)
        if target > 0:
            consistency_score = actual_value / target
        else:
            consistency_score = None

        result["per_habit"][habit] = {
            "goal": f"{target} {metric} per {period}",
            "metric": metric,
            "target_value": target,
            "actual_sessions": actual_sessions,
            "actual_minutes": actual_minutes,
            "consistency_score": consistency_score,
            "avg_rating": avg_rating,
            "avg_energy": avg_energy,
        }

    return json.dumps(result, indent=2)


def summarize_energy_patterns(tool_context: ToolContext, n_days: int) -> str:
    """
    Return JSON summarizing overall energy and rating patterns over last n_days.
    """
    db = _load_habit_db()
    user_db = db.get(USER_ID, {})

    today = datetime.today().date()
    from_date = today - timedelta(days=n_days - 1)
    stats = _aggregate_habits_over_range(user_db, from_date, today)

    all_ratings = []
    all_energies = []

    for s in stats.values():
        all_ratings.extend(s["ratings"])
        all_energies.extend(s["energies"])

    def safe_avg(lst: List[float]) -> float | None:
        return sum(lst) / len(lst) if lst else None

    result = {
        "window_days": n_days,
        "avg_rating_overall": safe_avg(all_ratings),
        "avg_energy_overall": safe_avg(all_energies),
        "habits": {},
    }

    for habit, s in stats.items():
        result["habits"][habit] = {
            "sessions": s["sessions"],
            "avg_rating": safe_avg(s["ratings"]),
            "avg_energy": safe_avg(s["energies"]),
        }

    return json.dumps(result, indent=2)


# Wrap tools as FunctionTool instances
add_habit_entry_tool = FunctionTool(add_habit_entry)
get_entries_by_date_tool = FunctionTool(get_entries_by_date)
get_entries_last_n_days_tool = FunctionTool(get_entries_last_n_days)
list_all_habits_tool = FunctionTool(list_all_habits)

set_goal_tool = FunctionTool(set_goal)
get_goals_tool = FunctionTool(get_goals)
analyze_progress_tool = FunctionTool(analyze_progress_against_goals)
summarize_energy_tool = FunctionTool(summarize_energy_patterns)

<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Agents</p>
</div>

In [37]:
# ======================
# 4. AGENTS
# ======================

# LoggingAgent: interpret logs + set goals

logging_instruction = """
You are LoggingAgent, a specialized agent that only handles:
- Logging daily habits
- Setting or updating goals

You have the following tools:
- add_habit_entry(date, habit, category, duration_minutes, rating, energy_level, notes)
- set_goal(habit, period, metric, target_value)
- list_all_habits()

Behavior rules:

1. When the user describes something they did (exercise, study, sleep, etc.):
   - Infer:
     - date (if not given, assume today's date in 'YYYY-MM-DD' from context)
     - habit name (e.g. 'workout', 'study', 'sleep', 'meditation')
     - category (e.g. 'health', 'learning', 'wellbeing', 'productivity')
     - duration in minutes (if mentioned, else null)
     - rating 1–5 (if mentioned like 'felt great' you may infer 4–5, 'felt bad' 1–2, else null)
     - energy_level 1–5 (if mentioned, else null)
     - notes (short free-text summary)
   - Call add_habit_entry with those values.
   - Then confirm to the user what you logged in 1–2 sentences.

2. When the user states a goal (e.g. 'I want to work out 4 times a week'):
   - Parse habit, period ('day', 'week', 'month', 'year'), metric ('sessions' or 'minutes'),
     and numeric target_value.
   - Call set_goal.
   - Then restate the goal to the user in friendly language.

3. For other queries (like summaries or plans), DO NOT try to answer them.
   Instead reply with something like:
   "I'm the logging specialist. Ask the Insight/Planner agent for analysis or planning."
"""

logging_agent = LlmAgent(
    name="LoggingAgent",
    model=DEFAULT_MODEL,
    description="Logs structured habit entries and goals.",
    instruction=logging_instruction,
    tools=[
        add_habit_entry_tool,
        set_goal_tool,
        list_all_habits_tool,
    ],
)


# InsightAgent: analyze trends & progress vs goals

insight_instruction = """
You are InsightAgent, a data analyst for habits.

You DO NOT log new entries or set goals. You only:
- Read existing habit data
- Read existing goals
- Analyze progress and patterns
- Return a structured JSON summary of the user's state

You have tools:
- get_entries_last_n_days(n_days)
- get_goals()
- analyze_progress_against_goals(n_days)
- summarize_energy_patterns(n_days)

Behavior:

1. When the user asks about:
   - their trends
   - consistency
   - how they are doing vs goals
   - energy or rating patterns
   - or asks for input to create a plan

   Do the following:

   a) Choose a suitable window (e.g. 7 or 30 days) or use the user's number if they say it.
   b) Call get_goals() to understand goals.
   c) Call analyze_progress_against_goals(n_days) to get numeric stats vs goals.
   d) Call summarize_energy_patterns(n_days) to understand ratings and energy.
   e) Optionally call get_entries_last_n_days(n_days) if you want raw context.

2. Combine tool outputs into a SINGLE JSON object with this shape:

{
  "window_days": <int>,
  "goals": { ... },                 // from get_goals()
  "progress_vs_goals": { ... },     // parsed from analyze_progress...
  "energy_insights": { ... },       // parsed from summarize_energy...
  "high_level_findings": [
    "...", "...", "..."             // 3–7 bullet-style textual insights
  ]
}

Return ONLY this JSON as your final answer (no extra commentary).
"""

insight_agent = LlmAgent(
    name="InsightAgent",
    model=DEFAULT_MODEL,
    description="Analyzes habits and goals and produces a JSON insight summary.",
    instruction=insight_instruction,
    tools=[
        get_entries_last_n_days_tool,
        get_goals_tool,
        analyze_progress_tool,
        summarize_energy_tool,
    ],
)


# PlannerAgent: generate personalized plan from summary JSON
planner_instruction = """
You are PlannerCoachAgent, a personalized habit coach.

You receive:
- A JSON analysis summary of the user's habits, goals, and energy patterns.
- A planning request from the user (e.g., plan for next week/month/year).

Your job:
- Read and understand the JSON.
- Create a realistic, personalized plan that helps the user move toward their goals,
  based on THEIR actual consistency, ratings, and energy patterns.

Guidelines:
- Be realistic: do not suddenly 5x their current behavior.
- Focus on 2–4 key habits, not everything.
- Use what you know:
  - If consistency is low, suggest smaller, easier versions of the habit.
  - If energy is higher at certain times, schedule hard tasks there.
  - If ratings are low, suggest experiments to make the habit easier or more enjoyable.
- Structure your answer clearly, for example:

PLAN SCOPE: next 7 days

1. Habit A (goal vs current, recommended schedule)
   - Concrete actions...
   - Suggested time of day + triggers ("after lunch", "before bed", etc.)
   - Simple fallback version for bad days.

2. Habit B ...

3. General guidance
   - 2–3 bullet points.

Avoid generic self-help tips; tie everything explicitly to the JSON summary and the user's patterns.
"""

planner_agent = LlmAgent(
    name="PlannerCoachAgent",
    model=DEFAULT_MODEL,
    description="Turns analysis JSON into a concrete, realistic plan.",
    instruction=planner_instruction,
)

<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Runners and helpers</p>
</div>

In [38]:
# ======================
# 5. Runners & helpers
# ======================

logging_runner = InMemoryRunner(agent=logging_agent, app_name=APP_NAME_LOGGING)
insight_runner = InMemoryRunner(agent=insight_agent, app_name=APP_NAME_INSIGHT)
planner_runner = InMemoryRunner(agent=planner_agent, app_name=APP_NAME_PLANNER)


async def ensure_session(runner: InMemoryRunner, session_name: str) -> str:
    """Create the session if needed and return its id (we just use session_name)."""
    session_service = runner.session_service
    try:
        await session_service.create_session(
            app_name=runner.app_name,
            user_id=USER_ID,
            session_id=session_name,
        )
    except Exception:
        # Already exists
        pass
    return session_name


async def run_agent_dialog(
    runner: InMemoryRunner,
    agent_name: str,
    user_messages: List[str] | str,
    session_name: str,
    capture_last: bool = False,
) -> str | None:
    """
    Simple helper to send one or more messages to a specific agent and print responses.
    If capture_last=True, returns the last non-empty text from the agent.
    """
    session_id = await ensure_session(runner, session_name)

    if isinstance(user_messages, str):
        user_messages = [user_messages]

    last_text: str | None = None

    for msg in user_messages:
        print(f"\n[{agent_name} session={session_name}]")
        print(f"User > {msg}")

        content = types.Content(
            role="user",
            parts=[types.Part(text=msg)],
        )

        async for event in runner.run_async(
            user_id=USER_ID,
            session_id=session_id,
            new_message=content,
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    text = getattr(part, "text", None)
                    if text and text != "None":
                        print(f"{agent_name} > {text}")
                        last_text = text

    return last_text if capture_last else None

<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Demo</p>
</div>


In [42]:
# ======================
# 6. Demo workflow – logging -> insight -> plan
# ======================

async def demo_multi_agent_flow():
    today_str = datetime.today().date().isoformat()

    # 1) Use LoggingAgent to log some synthetic data & set goals
    await run_agent_dialog(
        logging_runner,
        agent_name="LoggingAgent",
        user_messages=[
            f"I did a 30 minute workout today ({today_str}) and felt pretty good.",
            f"I studied DSA for 45 minutes today but was a bit tired.",
            f"I meditated for 10 minutes today and felt calm.",
            "I want to work out 4 times a week.",
            "I want to study 10 hours per week.",
        ],
        session_name="logging_session",
    )

    # 2) Use InsightAgent to get a JSON summary for last 7 days
    insight_json = await run_agent_dialog(
        insight_runner,
        agent_name="InsightAgent",
        user_messages="Using your tools, analyze my last 7 days and reply ONLY with the JSON object described in your instructions.",
        session_name="insight_session",
        capture_last=True,
    )

    # 3) Use PlannerCoachAgent to turn that JSON into a concrete plan
    if insight_json:
        planning_request = f"""
Here is your analysis summary JSON:

{insight_json}

User request: Based on this summary, create a realistic plan for the next 7 days to move me closer to my goals.
"""
        await run_agent_dialog(
            planner_runner,
            agent_name="PlannerCoachAgent",
            user_messages=planning_request,
            session_name="planner_session",
        )
    else:
        print("No insight JSON captured; planner step skipped.")


await demo_multi_agent_flow()


[LoggingAgent session=logging_session]
User > I did a 30 minute workout today (2025-12-01) and felt pretty good.
LoggingAgent > I've already logged this for you. Please let me know if there's anything else!

[LoggingAgent session=logging_session]
User > I studied DSA for 45 minutes today but was a bit tired.
LoggingAgent > I've already logged this for you. Please let me know if there's anything else!

[LoggingAgent session=logging_session]
User > I meditated for 10 minutes today and felt calm.
LoggingAgent > I've already logged this for you. Please let me know if there's anything else!

[LoggingAgent session=logging_session]
User > I want to work out 4 times a week.
LoggingAgent > I've already set this goal for you. Please let me know if there's anything else!

[LoggingAgent session=logging_session]
User > I want to study 10 hours per week.
LoggingAgent > I've already set this goal for you. Please let me know if there's anything else!

[InsightAgent session=insight_session]
User > Usi

<div style="border: 2px solid black; padding: 10px; border-radius: 5px; background-color: lightyellow;">
<p>Evaluation Set</p>
</div>

In [40]:
EVAL_TESTS = [
    # Logging agent tests
    {
        "agent": "LoggingAgent",
        "session": "eval_logging",
        "message": "I did a 20 minute workout today and felt great.",
        "description": "Should log a workout entry with duration ~20, positive rating/energy."
    },
    {
        "agent": "LoggingAgent",
        "session": "eval_logging",
        "message": "I want to study 10 hours a week.",
        "description": "Should set a study goal of ~600 minutes/week."
    },

    # Insight agent test
    {
        "agent": "InsightAgent",
        "session": "eval_insight",
        "message": "Analyze my habits for the last 7 days.",
        "description": "Should return a valid JSON analysis summary."
    },

    # Planner agent test
    {
        "agent": "PlannerCoachAgent",
        "session": "eval_planner",
        "message": """
Here is a sample summary JSON:
{
  "window_days": 7,
  "progress_vs_goals": {
    "workout": {
      "metric": "sessions",
      "target_value": 4,
      "actual_sessions": 2,
      "consistency_score": 0.5
    }
  },
  "energy_insights": {
    "avg_energy_overall": 3.2
  },
  "high_level_findings": ["Workout improving but below goal"]
}
Based on this summary, create a realistic 7-day plan.
""",
        "description": "Should output a concrete, personalized 7-day plan."
    }
]


async def run_evaluation():
    print("===== Evaluation Start =====")
    for test in EVAL_TESTS:
        print("\n--------------------------------------")
        print(f"Test: {test['description']}\n")

        if test["agent"] == "LoggingAgent":
            await run_agent_dialog(logging_runner, "LoggingAgent",
                                   test["message"], test["session"])

        elif test["agent"] == "InsightAgent":
            await run_agent_dialog(insight_runner, "InsightAgent",
                                   test["message"], test["session"])

        elif test["agent"] == "PlannerCoachAgent":
            await run_agent_dialog(planner_runner, "PlannerCoachAgent",
                                   test["message"], test["session"])
    print("\n===== Evaluation Complete =====")


In [41]:
await run_evaluation()

===== Evaluation Start =====

--------------------------------------
Test: Should log a workout entry with duration ~20, positive rating/energy.


[LoggingAgent session=eval_logging]
User > I did a 20 minute workout today and felt great.



--------------------------------------
Test: Should set a study goal of ~600 minutes/week.


[LoggingAgent session=eval_logging]
User > I want to study 10 hours a week.



--------------------------------------
Test: Should return a valid JSON analysis summary.


[InsightAgent session=eval_insight]
User > Analyze my habits for the last 7 days.



--------------------------------------
Test: Should output a concrete, personalized 7-day plan.


[PlannerCoachAgent session=eval_planner]
User > 
Here is a sample summary JSON:
{
  "window_days": 7,
  "progress_vs_goals": {
    "workout": {
      "metric": "sessions",
      "target_value": 4,
      "actual_sessions": 2,
      "consistency_score": 0.5
    }
  },
  "energy_insights": {
    "avg_energy_overall": 3.2
  },
  "high_level_findings": ["Workout improving but below goal"]
}
Based on this summary, create a realistic 7-day plan.

PlannerCoachAgent > PLAN SCOPE: next 7 days

This plan focuses on your "workout" habit, aiming to increase your consistency from 50% (2 sessions out of a goal of 4) towards your target.

1.  **Workout Sessions**
    *   **Goal:** 4 sessions this week.
    *   **Current Consistency:** 50% (2 sessions/week).
    *   **Recommended Schedule:** Aim for **3 sessions** this week. This is a realistic step up from your current 2 sessions, focusing on building co